<a href="https://colab.research.google.com/github/StevenKim1105/StevenKim1105/blob/main/tutorial_streamflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MHPI tutorial

This code contains deep learning code used to model hydrologic systems from soil moisture to streamflow or from projection to forecast.

[![PyPI](https://img.shields.io/badge/pypi-version%200.1-blue)](https://pypi.org/project/hydroDL/0.1.0/)  [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3993880.svg)](https://doi.org/10.5281/zenodo.3993880) [![CodeStyle](https://img.shields.io/badge/code%20style-Black-black)]()


Welcome to our hydroDL tutorial at The Pennsylvania State University! The following notebook is designed to provide a quick start to our project and get you ready to write your own neural networks.

### git repo

In [2]:
import os
os.chdir("/content/")
!rm -rf hydroDL
!git clone https://github.com/mhpi/hydroDL.git
!mv hydroDL/hydroDL/* hydroDL

Cloning into 'hydroDL'...
remote: Enumerating objects: 1046, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 1046 (delta 46), reused 19 (delta 10), pack-reused 960
Receiving objects: 100% (1046/1046), 60.67 MiB | 22.04 MiB/s, done.
Resolving deltas: 100% (429/429), done.


### download data


In [3]:
!wget 'https://gdex.ucar.edu/dataset/camels/file/basin_timeseries_v1p2_metForcing_obsFlow.zip' -O '/content/basin_timeseries_v1p2_metForcing_obsFlow.zip'
!mkdir '/content/camels_attributes_v2.0/'
!mkdir '/content/camels_attributes_v2.0/camels_attributes_v2.0/'
!wget 'https://gdex.ucar.edu/dataset/camels/file/camels_attributes_v2.0.xlsx' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_attributes_v2.0.xlsx'
!wget 'https://gdex.ucar.edu/dataset/camels/file/camels_clim.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_clim.txt'
!wget 'https://gdex.ucar.edu/dataset/camels/file/camels_geol.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_geol.txt'
!wget 'https://gdex.ucar.edu/dataset/camels/file/camels_hydro.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_hydro.txt'
!wget 'https://gdex.ucar.edu/dataset/camels/file/camels_name.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_name.txt'
!wget 'https://gdex.ucar.edu/dataset/camels/file/camels_soil.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_soil.txt'
!wget 'https://gdex.ucar.edu/dataset/camels/file/camels_topo.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_topo.txt'
!wget 'https://gdex.ucar.edu/dataset/camels/file/camels_soil.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_soil.txt'
!wget 'https://gdex.ucar.edu/dataset/camels/file/camels_vege.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_vege.txt'

!unzip '/content/basin_timeseries_v1p2_metForcing_obsFlow.zip' -d '/content/basin_timeseries_v1p2_metForcing_obsFlow'


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/hru_forcing/daymet/09/05131500_hru_01602_cida_forcing_leap.txt  
  inflating: /content/basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/hru_forcing/daymet/09/05131500_hru_00859_cida_forcing_leap.txt  
  inflating: /content/basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/hru_forcing/daymet/09/05120500_hru_00836_cida_forcing_leap.txt  
  inflating: /content/basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/hru_forcing/daymet/09/05057000_hru_00537_cida_forcing_leap.txt  
  inflating: /content/basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/hru_forcing/daymet/09/05129115_hru_00526_cida_forcing_leap.txt  
  inflating: /content/basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/hru_forcing/daymet/09/05129115_hru_01610_cida_forcing_leap.txt  
  inflating: /content/basin_timeseries_v

In [4]:
# import Libraries

import os
import sys
os.chdir("/content/hydroDL")
sys.path.append('..')

import torch
import random
import numpy as np
import pandas as pd

sys.path.append('../')
from hydroDL import master
from hydroDL.master import default
from hydroDL.master.master import loadModel, wrapMaster, writeMasterFile
from hydroDL.master.master import readMasterFile
from hydroDL.post import stat
from hydroDL.data import camels
from hydroDL.model.test import testModel
from hydroDL.model.train import trainModel
from hydroDL.model.rnn import CudnnLstmModel, CpuLstmModel
from hydroDL.model.crit import RmseLoss
from hydroDL.utils.norm import re_folder

loading package hydroDL


/content/hydroDL/data/camels.py:64: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if s is "name":
/content/hydroDL/data/camels.py:131: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if dataset is "daymet":
/content/hydroDL/data/camels.py:133: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif dataset is "nldas_extended":
/content/hydroDL/data/camels.py:561: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if opt is "all":


In [5]:
# set configuration
train_date = [19851001, 19951001]  # Training period
input_path = "/content/"
camels.initcamels(input_path)  # initialize three camels module-scope variables in camels.py
output_path = "./output/streamflow/"  

# time series variables list
var_time_series = ['Dayl', 'PRCP', 'SRAD', 'Tmax', 'Tmin', 'Vp']
# constant variables list
var_constant = ['elev_mean', 'slope_mean', 'area_gages2', 'frac_forest', 'lai_max',
         'lai_diff', 'dom_land_cover_frac', 'dom_land_cover', 'root_depth_50',
         'soil_depth_statsgo', 'soil_porosity', 'soil_conductivity', 'max_water_content',
         'geol_1st_class', 'geol_2nd_class', 'geol_porostiy', 'geol_permeability']
# target variable list
target = ['Streamflow']

# generate output folder
re_folder(output_path)

read usgs streamflow 13.313788890838623
read usgs streamflow 10.073385000228882


In [6]:
# hyperparameter
EPOCH = 10
BATCH_SIZE = 100
RHO = 365
HIDDENSIZE = 256
SEED = 42

In [7]:
# Fix random seed
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [8]:
# load your datasets
"""
You can change it with your data. The data structure is as follows:
train_x (forcing data, e.g. precipitation, temperature ...): [pixels, time, features] 
train_c (constant data, e.g. soil properties, land cover ...): [pixels, features]
target/train_y (e.g. soil moisture, streamflow ...): [pixels, time, 1]
Data type: numpy.float
"""
# define dataset
opt_data = default.optDataCamels
opt_data = default.update(opt_data, varT=camels.forcingLst, varC=camels.attrLstSel, tRange=train_date)
train_loader = camels.DataframeCamels(subset=opt_data["subset"], tRange=opt_data["tRange"])
train_x = train_loader.getDataTs(varLst=opt_data["varT"], doNorm=False, rmNan=False)
train_y = train_loader.getDataObs(doNorm=False, rmNan=False, basinnorm=False)
y_temp = camels.basinNorm(train_y, opt_data["subset"], toNorm=True)
train_c = train_loader.getDataConst(varLst=opt_data["varC"], doNorm=False, rmNan=False)
# process, do normalization and remove nan
series_data = np.concatenate([train_x, y_temp], axis=2)
series_var_list = camels.forcingLst + ["runoff"]
# calculate statistics for norm
stat_dict = camels.getStatDic(
    attrLst=camels.attrLstSel,
    attrdata=train_c,
    seriesLst=series_var_list,
    seriesdata=series_data,)
# normalize
attr_norm = camels.transNormbyDic(train_c, camels.attrLstSel, stat_dict, toNorm=True)
attr_norm[np.isnan(attr_norm)] = 0.0
series_norm = camels.transNormbyDic(
    series_data, series_var_list, stat_dict, toNorm=True)

# prepare the inputs
train_x = series_norm[:, :, :-1]  # forcing, not include obs
train_x[np.isnan(train_x)] = 0.0
train_y = np.expand_dims(series_norm[:, :, -1], 2)
train_c = attr_norm


read usgs streamflow 10.021795511245728
read usgs streamflow 12.108990669250488


In [9]:
# define model and update configure
if torch.cuda.is_available():
    opt_model = default.optLstm
else:
    opt_model = default.update(default.optLstm, name="hydroDL.model.rnn.CpuLstmModel")
opt_model = default.update(default.optLstm, hiddenSize=HIDDENSIZE)
# define loss function
opt_loss = default.optLossRMSE
# define training options
opt_train = default.update(default.optTrainCamels, miniBatch=[BATCH_SIZE, RHO], nEpoch=EPOCH, saveEpoch=1, seed=SEED,)

opt_all = wrapMaster(output_path, opt_data, opt_model, opt_loss, opt_train)
nx = train_x.shape[-1] + train_c.shape[-1]  # update nx, nx = nx + nc
ny = train_y.shape[-1]
# load model for training
if torch.cuda.is_available():
    model = CudnnLstmModel(nx=nx, ny=ny, hiddenSize=HIDDENSIZE)
else:
    model = CpuLstmModel(nx=nx, ny=ny, hiddenSize=HIDDENSIZE)
opt_model = default.update(opt_model, nx=nx, ny=ny)

loss_fn = RmseLoss()
# update and write the dictionary variable to out folder for logging and future testing
opt_all = wrapMaster(output_path, opt_data, opt_model, opt_loss, opt_train)
writeMasterFile(opt_all)

write master file ./output/streamflow/master.json


'./output/streamflow/'

In [10]:
# training the model
model = trainModel(
    model,
    train_x,
    train_y,
    train_c,
    loss_fn,
    nEpoch=EPOCH,
    miniBatch=[BATCH_SIZE, RHO],
    saveEpoch=1,
    saveFolder=output_path,
)

Training CudnnLstmModel:   0%|          | 0/10 [00:00<?, ?it/s]/content/hydroDL/model/rnn/CudnnLstm.py:104: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
  output, hy, cy, reserve, new_weight_buf = torch._cudnn_rnn(
Training CudnnLstmModel:  70%|███████   | 7/10 [02:17<00:58, 19.60s/it, loss=0.465]


KeyboardInterrupt: ignored

In [ ]:
# validate the result
# load validation datasets
# load your data. same as training data
valid_epoch = EPOCH  # choose the model to test after trained "valid_epoch" epoches
subset = "All"  # 'All': use all the CAMELS gages to test; Or pass the gage list
valid_date = [19951001, 20051001]  # Testing period
valid_batch = 100  # do batch forward to save GPU memory
# load testing data
mDict = readMasterFile(output_path)
opt_data = mDict["data"]
valid_loader = camels.DataframeCamels(subset=subset, tRange=valid_date)
valid_x = valid_loader.getDataTs(varLst=opt_data["varT"], doNorm=False, rmNan=False)
obs = valid_loader.getDataObs(doNorm=False, rmNan=False, basinnorm=False)
valid_c = valid_loader.getDataConst(varLst=opt_data["varC"], doNorm=False, rmNan=False)

# do normalization and remove nan
series_var_list = opt_data["varT"]
attrLst = opt_data["varC"]
attr_norm = camels.transNormbyDic(valid_c, attrLst, stat_dict, toNorm=True)
attr_norm[np.isnan(attr_norm)] = 0.0
valid_x = camels.transNormbyDic(valid_x, series_var_list, stat_dict, toNorm=True)
valid_x[np.isnan(valid_x)] = 0.0
valid_c = attr_norm


read master file ./output/streamflow/master.json
read usgs streamflow 9.97184705734253
read usgs streamflow 12.417492866516113
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_topo.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_clim.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_hydro.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_vege.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_soil.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_geol.txt


In [ ]:
# load and forward the model for validation
test_model = loadModel(output_path, epoch=valid_epoch)
filePathLst = master.master.namePred(
    output_path, valid_date, "All", epoch=valid_epoch
)  # prepare the name of csv files to save testing results
testModel(test_model, valid_x, c=valid_c, batchSize=valid_batch, filePathLst=filePathLst)

pred = pd.read_csv(filePathLst[0], dtype=np.float, header=None).values[:, :, None]
# transform back to the original observation
temppred = camels.transNormbyDic(pred, "runoff", stat_dict, toNorm=False)
pred = camels.basinNorm(temppred, subset, toNorm=False)

# change the units ft3/s to m3/s
obs = obs * 0.0283168
pred = pred * 0.0283168

# calculate statistic metrics
statDictLst = stat.statError(pred.squeeze(), obs.squeeze())
print(np.nanmedian(statDictLst["NSE"]))

read master file ./output/streamflow/master.json


<ipython-input-15-7395e3537595>:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred = pd.read_csv(filePathLst[0], dtype=np.float, header=None).values[:, :, None]


/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_topo.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_clim.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_hydro.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_vege.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_soil.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_geol.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_topo.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_clim.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_hydro.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_vege.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_soil.txt
/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_geol.txt


/content/hydroDL/post/stat.py:56: RuntimeWarning: divide by zero encountered in double_scalars
  PBiaslow[k] = np.sum(lowpred - lowtarget) / np.sum(lowtarget) * 100


0.5008026627737916
